In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from pandas import to_datetime
import matplotlib.pyplot as plt
import time
import seaborn as sns
import pmdarima as pm
from pmdarima.model_selection import train_test_split


from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.metrics import r2_score

import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm
plt.rcParams['font.family'] ='Malgun Gothic'
plt.rcParams['axes.unicode_minus'] =False

In [2]:
df = pd.read_csv("C:/Users/User/github/Quant/data/kospi_category.csv", encoding='cp949')
start_date = '2004-01'
end_date = '2020-03'
 
df['time'] = pd.date_range(start_date,end_date,freq='m')
df.drop('date',axis=1, inplace=True)
df.set_index('time', inplace=True) 

In [3]:
# train 데이터와 validation 데이터 나누기.
X_train = df[df.index < '2019-01-01']
X_valid = df[df.index >= '2019-01-01']

# 각 데이터의 사이즈 확인하기
print('X_train Shape', X_train.shape)
print('X_Valid Shape', X_valid.shape)

X_train Shape (180, 22)
X_Valid Shape (14, 22)


# Moving Window X
# Auto-ARIMA 돌리기 - 계측값이 일별이면 m=1, 월별이면 m=12, 주별이면 m=52, 
# 계절성이 있는 데이터면 seasonal=True 로 바꿔야함. 알아서 d 값을 찾아줌.
arima_result = []
for i in tqdm(range(len(df.columns))):
    auto_arima_model = pm.auto_arima(y = X_train.iloc[:,i],
              start_P=0,
              max_p= 3,
              start_q = 0,
              max_q = 3,
              m = 12,
              seasonal=True,
              stepwise= False,
              trace= True)
    
    fcast2 = auto_arima_model.predict(14) 
    fcast2 = pd.Series(fcast2, index = X_valid.index)
    fcast2 = fcast2.rename("Auto Arima")
    fig, ax = plt.subplots(figsize=(15,5))
    chart = sns.lineplot(data = pd.DataFrame(X_train.iloc[:,i]))
    
    chart.set_title(df.columns[i]+' Auto Arima')
    
    fcast2.plot(ax=ax, color='red', marker="o")
    plt.plot(X_valid.iloc[:,i], color = 'blue', marker = 'o')
    plt.title(df.columns[i])
    plt.legend()
    plt.savefig("C:/Users/User/github/Quant/data/"+df.columns[i]+" AutoArima_seasonal_O.png")
    print(df.columns[i]+' The MSE of auto-arima is:', mean_squared_error(X_valid.iloc[:,i].values, fcast2.values))
    arima_result.append( mean_squared_error(X_valid.iloc[:,i].values, fcast2.values))
    

In [40]:
# Moving Window 적용
# Auto-ARIMA 돌리기 - 계측값이 일별이면 m=1, 월별이면 m=12, 주별이면 m=52, 
# 계절성이 있는 데이터면 seasonal=True 로 바꿔야함. 알아서 d 값을 찾아줌
arima_result = []
arima_result_2 = []
arima_result_3 = []
arima_val = []
for i in tqdm(range(len(df.columns))):
    predictions = list()
    history = [x for x in X_train.iloc[:,i]]
    for t in range(len(X_valid)):
        model = pm.auto_arima(y = history)
        output = model.predict()
        yhat = output[0]
        predictions.append(yhat)
        obs = X_valid.iloc[:,i][t]
        history.append(obs)
    
    predictions = pd.Series(predictions, index = X_valid.index)

    chart = sns.lineplot(data = pd.DataFrame(X_train.iloc[:,i]))
    chart.set_title(df.columns[i]+' Auto Arima')

    plt.plot(X_train.iloc[:,i], color='blue')
    plt.plot(predictions, color = 'red')
    plt.plot(X_valid.iloc[:,i], color = 'blue')
    plt.title(df.columns[i])
    plt.legend()
    plt.savefig("C:/Users/User/github/Quant/data/"+df.columns[i]+" moving_window.png")
    print(df.columns[i]+' The MSE of auto-arima is:', mean_squared_error(X_valid.iloc[:,i].values, predictions))
    arima_result.append( mean_squared_error(X_valid.iloc[:,i].values, predictions))
    arima_result_2.append(model.summary())
    arima_result_3.append(r2_score(X_valid.iloc[:,i], predictions))
    arima_val.append(predictions)
    plt.clf()
    

  5%|▍         | 1/22 [00:03<01:12,  3.45s/it]

제조업 The MSE of auto-arima is: 69869.70982860656


  9%|▉         | 2/22 [00:09<01:38,  4.90s/it]

음식료품 The MSE of auto-arima is: 27459.132002179493


 14%|█▎        | 3/22 [00:27<03:31, 11.15s/it]

...

 95%|█████████▌| 21/22 [02:37<00:06,  6.79s/it]

보험 The MSE of auto-arima is: 837589.1403827695


100%|██████████| 22/22 [02:41<00:00,  7.34s/it]

서비스업 The MSE of auto-arima is: 1226.7039571428556


<Figure size 640x480 with 0 Axes>

In [5]:
arima_result

[66547.44699062365,
 29202.670873225226,
 399.25049185750674,
 332.15399226545077,
 58218.444877320595,
 765504.4782686004,
 7607.996171202804,
 72543.59124172278,
 2601.4623567576245,
 1235285.477114776,
 74071.9941619176,
 10100.458705821778,
 444.0328650502991,
 4172.2882437212165,
 74.25638861887967,
 4857.592235572085,
 165.40785619799104,
 575.4969089000894,
 193.73748391009153,
 11034.944497536722,
 977905.2618693763,
 1392.7647770822573]

In [6]:
arima_result_2[0]

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                  193
Model:               SARIMAX(0, 1, 0)   Log Likelihood               -1268.866
Date:                Tue, 03 Jan 2023   AIC                           2541.733
Time:                        13:00:30   BIC                           2548.248
Sample:                             0   HQIC                          2544.371
                                - 193                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept     19.1607     13.865      1.382      0.167      -8.014      46.336
sigma2      3.219e+04   2496.112     12.897      0.000    2.73e+04    3.71e+04
===================================================================================
Ljung-Box (L1) (Q):                   0.17   Jarque-Bera (JB):                47.39
Prob(Q):                              0.68   Prob(JB):                         0.00
Heteroskedasticity (H):               1.48   Skew:                            -0.71
Prob(H) (two-sided):                  0.12   Kurtosis:                         4.97
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [7]:
arima_result_2[1]

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                  193
Model:               SARIMAX(0, 1, 0)   Log Likelihood               -1263.368
Date:                Tue, 03 Jan 2023   AIC                           2530.736
Time:                        13:04:45   BIC                           2537.251
Sample:                             0   HQIC                          2533.375
                                - 193                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept     10.3835     13.305      0.780      0.435     -15.694      36.461
sigma2       3.04e+04   2858.822     10.634      0.000    2.48e+04     3.6e+04
===================================================================================
Ljung-Box (L1) (Q):                   0.08   Jarque-Bera (JB):                12.12
Prob(Q):                              0.77   Prob(JB):                         0.00
Heteroskedasticity (H):               2.23   Skew:                            -0.53
Prob(H) (two-sided):                  0.00   Kurtosis:                         3.63
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [8]:
arima_result_2[2]

<class 'statsmodels.iolib.summary.Summary'>
"""
                                      SARIMAX Results                                      
===========================================================================================
Dep. Variable:                                   y   No. Observations:                  193
Model:             SARIMAX(2, 1, 2)x(1, 0, [], 12)   Log Likelihood                -809.033
Date:                             Tue, 03 Jan 2023   AIC                           1632.066
Time:                                     13:08:51   BIC                           1654.869
Sample:                                          0   HQIC                          1641.301
                                             - 193                                         
Covariance Type:                               opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.9380      1.388      0.676      0.499      -1.783       3.659
ar.L1          0.6802      0.110      6.184      0.000       0.465       0.896
ar.L2         -0.5796      0.088     -6.592      0.000      -0.752      -0.407
ma.L1         -0.7617      0.066    -11.464      0.000      -0.892      -0.631
ma.L2          0.8354      0.060     13.946      0.000       0.718       0.953
ar.S.L12      -0.1471      0.088     -1.666      0.096      -0.320       0.026
sigma2       266.3257     21.009     12.677      0.000     225.150     307.502
===================================================================================
Ljung-Box (L1) (Q):                   0.62   Jarque-Bera (JB):               116.90
Prob(Q):                              0.43   Prob(JB):                         0.00
Heteroskedasticity (H):               3.30   Skew:                             0.32
Prob(H) (two-sided):                  0.00   Kurtosis:                         6.77
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [9]:
arima_result_2[3]

<class 'statsmodels.iolib.summary.Summary'>
"""
                                        SARIMAX Results                                        
===============================================================================================
Dep. Variable:                                       y   No. Observations:                  193
Model:             SARIMAX(0, 1, 0)x(1, 0, [1, 2], 12)   Log Likelihood                -868.444
Date:                                 Tue, 03 Jan 2023   AIC                           1746.887
Time:                                         13:12:48   BIC                           1763.175
Sample:                                              0   HQIC                          1753.484
                                                 - 193                                         
Covariance Type:                                   opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      1.3492      2.938      0.459      0.646      -4.409       7.108
ar.S.L12      -0.2703      0.273     -0.992      0.321      -0.804       0.264
ma.S.L12       0.4117      0.266      1.548      0.122      -0.110       0.933
ma.S.L24       0.3449      0.092      3.768      0.000       0.166       0.524
sigma2       489.2893     40.553     12.066      0.000     409.808     568.771
===================================================================================
Ljung-Box (L1) (Q):                   0.52   Jarque-Bera (JB):                34.60
Prob(Q):                              0.47   Prob(JB):                         0.00
Heteroskedasticity (H):               0.91   Skew:                            -0.51
Prob(H) (two-sided):                  0.70   Kurtosis:                         4.82
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [10]:
arima_result_2[4]

<class 'statsmodels.iolib.summary.Summary'>
"""
                                      SARIMAX Results                                       
============================================================================================
Dep. Variable:                                    y   No. Observations:                  193
Model:             SARIMAX(2, 1, 2)x(0, 0, [1], 12)   Log Likelihood               -1318.835
Date:                              Tue, 03 Jan 2023   AIC                           2651.671
Time:                                      13:16:50   BIC                           2674.473
Sample:                                           0   HQIC                          2660.906
                                              - 193                                         
Covariance Type:                                opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept     16.9941     16.227      1.047      0.295     -14.811      48.799
ar.L1          0.6899      0.166      4.168      0.000       0.365       1.014
ar.L2         -0.6366      0.168     -3.782      0.000      -0.966      -0.307
ma.L1         -0.7240      0.138     -5.260      0.000      -0.994      -0.454
ma.L2          0.8005      0.141      5.673      0.000       0.524       1.077
ma.S.L12      -0.2036      0.080     -2.539      0.011      -0.361      -0.046
sigma2      5.396e+04   4068.472     13.262      0.000     4.6e+04    6.19e+04
===================================================================================
Ljung-Box (L1) (Q):                   0.06   Jarque-Bera (JB):                51.41
Prob(Q):                              0.80   Prob(JB):                         0.00
Heteroskedasticity (H):               1.36   Skew:                            -0.34
Prob(H) (two-sided):                  0.22   Kurtosis:                         5.44
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [11]:
arima_result_2[5]

<class 'statsmodels.iolib.summary.Summary'>
"""
                                        SARIMAX Results                                        
===============================================================================================
Dep. Variable:                                       y   No. Observations:                  193
Model:             SARIMAX(0, 1, 0)x(1, 0, [1, 2], 12)   Log Likelihood               -1484.830
Date:                                 Tue, 03 Jan 2023   AIC                           2979.661
Time:                                         13:20:55   BIC                           2995.948
Sample:                                              0   HQIC                          2986.257
                                                 - 193                                         
Covariance Type:                                   opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept     52.5730     80.066      0.657      0.511    -104.354     209.500
ar.S.L12      -0.5430      0.170     -3.195      0.001      -0.876      -0.210
ma.S.L12       0.4997      0.172      2.911      0.004       0.163       0.836
ma.S.L24       0.3897      0.073      5.314      0.000       0.246       0.533
sigma2      2.964e+05   2.21e+04     13.397      0.000    2.53e+05     3.4e+05
===================================================================================
Ljung-Box (L1) (Q):                   1.08   Jarque-Bera (JB):               284.43
Prob(Q):                              0.30   Prob(JB):                         0.00
Heteroskedasticity (H):               9.40   Skew:                            -1.00
Prob(H) (two-sided):                  0.00   Kurtosis:                         8.62
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [46]:
arima_result_3

[-1.1325987038168823,
 0.6959492712246852,
 0.22920582194327566,
 0.5106843950365143,
 0.5865436332711326,
 0.14451717540418318,
 0.6658754386156813,
 0.6429501007583611,
 0.4152698570478346,
 0.2687078798438063,
 -0.2537026441638144,
 -0.656628980013449,
 0.623456686403431,
 0.6539082798294076,
 0.6673911005571664,
 0.11572690002134312,
 0.49186086012383323,
 0.6170265112025105,
 0.7462709471425942,
 0.2890123778115551,
 0.7882091948143657,
 -0.012694488123802916]

In [48]:
predict = pd.DataFrame(arima_val).T